<a href="https://colab.research.google.com/github/Nataliahfk/TELECOM-X---PARTE-2/blob/main/TelecomX_SegundaParte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**TELECOM X - PARTE 2: Previsión de cancelación en las suscripciones**

##**Descripción del proyecto:**

Desarrollo de modelos predictivos capaces de preveer que clientes tienen mayor probabilidad de cancelar sus suscripciones.

##**Objetivos:**

- Preparar los datos para el modelado (tratamiento, codificación, normalización).
- Realizar análisis de correlación y selección de variables.
- Entrenar dos o más modelos de clasificación.
- Evaluar el rendimiento de los modelos con métricas.
- Interpretar los resultados, incluyendo la importancia de las variables.
- Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.

#📊 **Preparar los datos para el modelado**

##1.1. Importando y conociendo los datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
#Importando datos
datos = pd.read_csv('/content/df_limpo.csv')
datos.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,Total.Day,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,2.20,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,2.01,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,2.34,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,3.17,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,2.97,83.9,267.40


In [3]:
datos.columns

Index(['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen',
       'customer.Partner', 'customer.Dependents', 'customer.tenure',
       'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService',
       'internet.OnlineSecurity', 'internet.OnlineBackup',
       'internet.DeviceProtection', 'internet.TechSupport',
       'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract',
       'account.PaperlessBilling', 'account.PaymentMethod', 'Total.Day',
       'account.Charges.Monthly', 'account.Charges.Total'],
      dtype='object')

In [4]:
# Cambiar el nombre de las columnas
datos.rename(columns={
    'customerID': 'id_cliente',
    'customer.gender': 'genero',
    'customer.SeniorCitizen': 'adulto_mayor',
    'customer.Partner': 'tiene_pareja',
    'customer.Dependents': 'tiene_dependientes',
    'customer.tenure': 'antiguedad_meses',
    'phone.PhoneService': 'servicio_telefonico',
    'phone.MultipleLines': 'lineas_multiples',
    'internet.InternetService': 'servicio_internet',
    'internet.OnlineSecurity': 'seguridad_en_linea',
    'internet.OnlineBackup': 'respaldo_en_linea',
    'internet.DeviceProtection': 'proteccion_dispositivo',
    'internet.TechSupport': 'soporte_tecnico',
    'internet.StreamingTV': 'streaming_tv',
    'internet.StreamingMovies': 'streaming_peliculas',
    'account.Contract': 'tipo_contrato',
    'account.PaperlessBilling': 'facturacion_electronica',
    'account.PaymentMethod': 'metodo_pago',
    'Total.Day': 'total_dia',
    'account.Charges.Monthly': 'cargo_mensual',
    'account.Charges.Total': 'cargo_total',
    'Churn': 'cancelacion'
}, inplace=True)

# 🛠️ Tratamiento de los Datos

✔️ Remover columnas irrelevantes

***Nota**: inicialmente, se eliminarán las varialbes id_cliente ya que esta no aporta poder predictivo debido a que es un simple identificador de cliente, y cuentas_diarias ya que esta variable fue agregada simplemente al dividir cargos_mensuales en 30 días, lo cual resulta redundante y no aporta ningun valor agregado al análisis.*

In [5]:
# Eliminación de Columnas Irrelevantes
datos.drop('id_cliente', axis=1, inplace=True)
datos.drop('total_dia', axis=1, inplace=True)
datos.head()

,cancelacion,genero,adulto_mayor,tiene_pareja,tiene_dependientes,antiguedad_meses,servicio_telefonico,lineas_multiples,servicio_internet,seguridad_en_linea,respaldo_en_linea,proteccion_dispositivo,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago,cargo_mensual,cargo_total
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [6]:
datos.shape[0]

7043

In [7]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cancelacion              7043 non-null   object 
 1   genero                   7043 non-null   object 
 2   adulto_mayor             7043 non-null   int64  
 3   tiene_pareja             7043 non-null   object 
 4   tiene_dependientes       7043 non-null   object 
 5   antiguedad_meses         7043 non-null   int64  
 6   servicio_telefonico      7043 non-null   object 
 7   lineas_multiples         7043 non-null   object 
 8   servicio_internet        7043 non-null   object 
 9   seguridad_en_linea       7043 non-null   object 
 10  respaldo_en_linea        7043 non-null   object 
 11  proteccion_dispositivo   7043 non-null   object 
 12  soporte_tecnico          7043 non-null   object 
 13  streaming_tv             7043 non-null   object 
 14  streaming_peliculas     

In [17]:
print("\nEstadísticas Variables númericas:")
datos.describe()


Estadísticas Variables númericas:


,adulto_mayor,antiguedad_meses,cargo_mensual,cargo_total
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [9]:
print("\nEstadísticas Variables númericas:")
datos.describe(include='object')


Estadísticas Variables númericas:


,cancelacion,genero,tiene_pareja,tiene_dependientes,servicio_telefonico,lineas_multiples,servicio_internet,seguridad_en_linea,respaldo_en_linea,proteccion_dispositivo,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,2,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4
top,No,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
freq,5174,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365


In [10]:
columnas_categoricas = datos.select_dtypes(include='object').columns
print(columnas_categoricas)

Index(['cancelacion', 'genero', 'tiene_pareja', 'tiene_dependientes',
       'servicio_telefonico', 'lineas_multiples', 'servicio_internet',
       'seguridad_en_linea', 'respaldo_en_linea', 'proteccion_dispositivo',
       'soporte_tecnico', 'streaming_tv', 'streaming_peliculas',
       'tipo_contrato', 'facturacion_electronica', 'metodo_pago'],
      dtype='object')


#1.2. Visualización de las variables categóricas sobre Cancelaciones de los usuarios.

In [19]:
px.histogram(datos,x='genero',text_auto=True,color='cancelacion',barmode='group')

In [20]:
px.histogram(datos,x='tiene_pareja',text_auto=True,color='cancelacion',barmode='group')

In [21]:
px.histogram(datos,x='tiene_dependientes',text_auto=True,color='cancelacion',barmode='group')

In [23]:
datos['adulto_mayor'].value_counts(normalize=True).round(2)*100

,proportion
adulto_mayor,
0,84.0
1,16.0


In [24]:
px.histogram(datos,x='adulto_mayor',text_auto=True,color='cancelacion',barmode='group')

In [25]:
px.histogram(datos,x='tipo_contrato',text_auto=True,color='cancelacion',barmode='group')

In [27]:
px.histogram(datos,x='metodo_pago',text_auto=True,color='cancelacion',barmode='group')

#1.3. Visualización de las variables cuantitativas sobre Cancelaciones de los usuarios.

#2. Tranformación de los datos

✔️ Agrupación de No y No service

✅ Resumen: Por qué agrupamos "No" y "No internet service"

🎯 Objetivo

Reducir multicolinealidad y simplificar los datos sin perder relevancia para el modelo predictivo.

---------------------------------------------------------------
⚠️ El problema original

   - Variables como OnlineSecurity, StreamingTV, etc., tenían tres categorías:

     - "Yes" → cliente usa el servicio
     - "No" → cliente tiene internet, pero no contrató el servicio
     - "No internet service" → cliente ni siquiera tiene internet, por lo tanto no puede usar el servicio

Esto generaba multicolinealidad perfecta al transformar estas categorías en dummies, lo que:
   - Creaba correlación 1.0 entre variables
   - Generaba VIF infinito
   - Comprometía la estabilidad y el rendimiento de los modelos

-----------------------------------------------------------------
✅ La solución: agrupar "No internet service" como "No"

  - Agrupamos "No internet service" como "No" para simplificar la variable:

   - "Yes" = significa que usa el servicio
   - "No" = significa que no usa el servicio
   
Esto reduce la dimensionalidad y evita multicolinealidad.

La información de que el cliente no tiene internet sigue estando en la variable InternetService.

In [11]:
datos.columns

Index(['cancelacion', 'genero', 'adulto_mayor', 'tiene_pareja',
       'tiene_dependientes', 'antiguedad_meses', 'servicio_telefonico',
       'lineas_multiples', 'servicio_internet', 'seguridad_en_linea',
       'respaldo_en_linea', 'proteccion_dispositivo', 'soporte_tecnico',
       'streaming_tv', 'streaming_peliculas', 'tipo_contrato',
       'facturacion_electronica', 'metodo_pago', 'cargo_mensual',
       'cargo_total'],
      dtype='object')

In [12]:
# para crear uno nuevo
datos_clean = datos.copy()

# === Etapa 1: Agrupar "No internet service" como "No"
cols_to_fix = ['servicio_internet', 'seguridad_en_linea',
               'respaldo_en_linea', 'proteccion_dispositivo', 'soporte_tecnico',
               'streaming_tv', 'streaming_peliculas']

for col in cols_to_fix:
    datos_clean[col] = datos_clean[col].replace('No internet service', 'No')

datos = datos_clean

In [13]:
# separo en variales explicativas que es la X y mi variable dependiente que es y
X = datos.drop('cancelacion', axis=1)
y = datos['cancelacion']

In [14]:
X

,genero,adulto_mayor,tiene_pareja,tiene_dependientes,antiguedad_meses,servicio_telefonico,lineas_multiples,servicio_internet,seguridad_en_linea,respaldo_en_linea,proteccion_dispositivo,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago,cargo_mensual,cargo_total
0,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,No,No,13,Yes,No,DSL,Yes,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90
7039,Male,0,Yes,No,22,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70
7040,Male,0,No,No,2,Yes,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7041,Male,0,Yes,Yes,67,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [15]:
y

,cancelacion
0,No
1,No
2,Yes
3,Yes
4,Yes
...,...
7038,No
7039,Yes
7040,No
7041,No


##**Encoding**

In [16]:
# === Etapa 2: One-hot encoding (sin dummy trap)
categorias_col=['cancelacion', 'genero', 'tiene_pareja', 'tiene_dependientes',
       'servicio_telefonico', 'lineas_multiples', 'servicio_internet',
       'seguridad_en_linea', 'respaldo_en_linea', 'proteccion_dispositivo',
       'soporte_tecnico', 'streaming_tv', 'streaming_peliculas',
       'tipo_contrato', 'facturacion_electronica', 'metodo_pago']

df_encoded = pd.get_dummies(datos, columns=categorias_col, drop_first=True)

df_encoded

,adulto_mayor,antiguedad_meses,cargo_mensual,cargo_total,cancelacion_Yes,genero_Male,tiene_pareja_Yes,tiene_dependientes_Yes,servicio_telefonico_Yes,lineas_multiples_No phone service,...,proteccion_dispositivo_Yes,soporte_tecnico_Yes,streaming_tv_Yes,streaming_peliculas_Yes,tipo_contrato_One year,tipo_contrato_Two year,facturacion_electronica_Yes,metodo_pago_Credit card (automatic),metodo_pago_Electronic check,metodo_pago_Mailed check
0,0,9,65.60,593.30,False,False,True,True,True,False,...,False,True,True,False,True,False,True,False,False,True
1,0,9,59.90,542.40,False,True,False,False,True,False,...,False,False,False,True,False,False,False,False,False,True
2,0,4,73.90,280.85,True,True,False,False,True,False,...,True,False,False,False,False,False,True,False,True,False
3,1,13,98.00,1237.85,True,True,True,False,True,False,...,True,False,True,True,False,False,True,False,True,False
4,1,3,83.90,267.40,True,False,True,False,True,False,...,False,True,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,13,55.15,742.90,False,False,False,False,True,False,...,False,True,False,False,True,False,False,False,False,True
7039,0,22,85.10,1873.70,True,True,True,False,True,False,...,False,False,False,True,False,False,True,False,True,False
7040,0,2,50.30,92.75,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
7041,0,67,67.85,4627.65,False,True,True,True,True,False,...,True,True,False,True,False,True,False,False,False,True
